This notebook is for Performance Evaluation for different GPU Accelerator

## 1. Accelerator: GPU T4 x 2
* each GPU's memory: 16G

In [1]:
from huggingface_hub import login

huggingface_key = "xxx...xxx"
login(huggingface_key)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 36.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.6 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=187315346 sha256=6ebfbdcbdd164f80278a954d29a1bc9d620130264215f0fb93374a6ab4e0a283
  Stored in directory: /root/.cache/pip/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn


In [3]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mixtral-8x7B-v0.1", 
    torch_dtype=torch.float16, 
    device_map="auto",
    offload_folder="offload"
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

prompt = "My favourite condiment is"
model_inputs = tokenizer([prompt], return_tensors="pt").to(device)

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### Generate 5 new tokens

In [6]:
output_sequences = []
for i in tqdm(range(4)):
    with torch.amp.autocast('cuda'):
        generated_ids = model.generate(**model_inputs, max_new_tokens=5, do_sample=True)
        output_sequences.append(generated_ids)

100%|██████████| 4/4 [1:17:18<00:00, 1159.69s/it]


In [10]:
output_sequences = torch.cat(output_sequences, dim=1)

decoded_output = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)[0]
print(decoded_output)

My favourite condiment is the one I cannot pron My favourite condiment is a toss up between must My favourite condiment is Sriracha hot My favourite condiment is hot sauce.  I


### Generate 10 new tokens

In [5]:
output_sequences = []
for i in tqdm(range(2)):
    with torch.amp.autocast('cuda'):
        generated_ids = model.generate(**model_inputs, max_new_tokens=10, do_sample=True)
        output_sequences.append(generated_ids)

  0%|          | 0/2 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
100%|██████████| 2/2 [1:17:16<00:00, 2318.05s/it]


In [6]:
output_sequences = torch.cat(output_sequences, dim=1)

decoded_output = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)[0]
print(decoded_output)

My favourite condiment is mustard. My favourite animal is a cow. My favourite condiment is mayonnaise and with summer fast approaching I am


### Generate 20 new tokens

In [5]:
output_sequences = []
for i in tqdm(range(1)):
    with torch.amp.autocast('cuda'):
        generated_ids = model.generate(**model_inputs, max_new_tokens=20, do_sample=True)
        output_sequences.append(generated_ids)

  0%|          | 0/1 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
100%|██████████| 1/1 [1:10:14<00:00, 4214.97s/it]


In [6]:
output_sequences = torch.cat(output_sequences, dim=1)

decoded_output = tokenizer.batch_decode(output_sequences, skip_special_tokens=True)[0]
print(decoded_output)

My favourite condiment is without a doubt a good ol’ fashioned salsa – you might have noticed the many recipes
